In [ ]:
import tensorflow as tf
from tensorflow import keras
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import classification_report
from sklearn.feature_extraction import image
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
nc_fy    = 'APP-x.1400.ERA-I.1982-2018.07.Arctic.1982-2009.nc'
nc_fm    = 'APP-x.1400.ERA-I.1982-2018.06.Arctic.1982-2009.nc'
nc_fay   = 'APP-x.1400.ERA-I.1982-2018.07.anom.wrt.2001-2018.Arctic.1982-2009.nc'
nc_fam   = 'APP-x.1400.ERA-I.1982-2018.06.anom.wrt.2001-2018.Arctic.1982-2009.nc'
h5_model = 'APP-x.1400.ERA-I.1982-2018.07.Arctic.1982-2009.5V.RAND.100epoch_leaky.h5'
pg_train = 'APP-x.1400.ERA-I.1982-2018.07.training.Arctic.1982-2009.5V.RAND.100epoch_leaky.png'
pg_predt = 'APP-x.1400.ERA-I.1982-2018.07.prediction.Arctic.1982-2009.5V.RAND.100epoch_leaky.png'

In [ ]:
# Starting CNN Model
batch_size = 128
epochs = 200
psize  = 11
ptrain = 0.90
cx     = int((psize-1)/2)
cy     = int((psize-1)/2)

In [ ]:
# input file for SIC Yr-1

nc_fidy  = Dataset(nc_fy, 'r')
time    = nc_fidy.variables['time'][:]

ysic0   = nc_fidy.variables['icecon'][:]
ythk0   = nc_fidy.variables['t2m'][:]
yvtn0   = nc_fidy.variables['v10'][:]
ytem0   = nc_fidy.variables['sst'][:]
yalb0   = nc_fidy.variables['fal'][:]

# input file for SIC, THK, Temperature and ALBEDO Mon-1

nc_fidm = Dataset(nc_fm, 'r')
msic0   = nc_fidm.variables['icecon'][:]
mthk0   = nc_fidm.variables['t2m'][:]
mvtn0   = nc_fidm.variables['v10'][:]
mtem0   = nc_fidm.variables['sst'][:]
malb0   = nc_fidm.variables['fal'][:]

# input file for SIC Anomaly, Yr-1

nc_fiday = Dataset(nc_fay, 'r')
aysic0   = nc_fiday.variables['icecon'][:]
aythk0   = nc_fiday.variables['t2m'][:]
ayvtn0   = nc_fiday.variables['v10'][:]
aytem0   = nc_fiday.variables['sst'][:]
ayalb0   = nc_fiday.variables['fal'][:]

# input file for SIC Anomaly, Mon-1

nc_fidam = Dataset(nc_fam, 'r')
amsic0   = nc_fidam.variables['icecon'][:]
amthk0   = nc_fidam.variables['t2m'][:]
amvtn0   = nc_fidam.variables['v10'][:]
amtem0   = nc_fidam.variables['sst'][:]
amalb0   = nc_fidam.variables['fal'][:]

In [ ]:
ysic1   = np.transpose(ysic0)
ythk1   = np.transpose(ythk0)
yvtn1   = np.transpose(yvtn0)
ytem1   = np.transpose(ytem0)
yalb1   = np.transpose(yalb0)

msic1   = np.transpose(msic0)
mthk1   = np.transpose(mthk0)
mvtn1   = np.transpose(mvtn0)
mtem1   = np.transpose(mtem0)
malb1   = np.transpose(malb0)

aysic1   = np.transpose(aysic0)
aythk1   = np.transpose(aythk0)
ayvtn1   = np.transpose(ayvtn0)
aytem1   = np.transpose(aytem0)
ayalb1   = np.transpose(ayalb0)

amsic1   = np.transpose(amsic0)
amthk1   = np.transpose(amthk0)
amvtn1   = np.transpose(amvtn0)
amtem1   = np.transpose(amtem0)
amalb1   = np.transpose(amalb0)


print("msic1.shape:", msic1.shape)
print("ysic1.shape:", ysic1.shape)

In [ ]:
print, time

In [ ]:
# Remove NaNs

Y_SIC2   = np.nan_to_num(ysic1[:,:,1:])
Y_time   = time[1:]

ysic2    = np.nan_to_num(ysic1[:,:,:-1])
ythk2    = np.nan_to_num(ythk1[:,:,:-1])
yvtn2    = np.nan_to_num(yvtn1[:,:,:-1])
ytem2    = np.nan_to_num(ytem1[:,:,:-1])
yalb2    = np.nan_to_num(yalb1[:,:,:-1])


msic2    = np.nan_to_num(msic1[:,:,1:])
mthk2    = np.nan_to_num(mthk1[:,:,1:])
mvtn2    = np.nan_to_num(mvtn1[:,:,1:])
mtem2    = np.nan_to_num(mtem1[:,:,1:])
malb2    = np.nan_to_num(malb1[:,:,1:])

aysic2    = np.nan_to_num(aysic1[:,:,:-1])
aythk2    = np.nan_to_num(aythk1[:,:,:-1])
ayvtn2    = np.nan_to_num(ayvtn1[:,:,:-1])
aytem2    = np.nan_to_num(aytem1[:,:,:-1])
ayalb2    = np.nan_to_num(ayalb1[:,:,:-1])

amsic2    = np.nan_to_num(amsic1[:,:,1:])
amthk2    = np.nan_to_num(amthk1[:,:,1:])
amvtn2    = np.nan_to_num(amvtn1[:,:,1:])
amtem2    = np.nan_to_num(amtem1[:,:,1:])
amalb2    = np.nan_to_num(amalb1[:,:,1:])

In [ ]:
print('input data of SIC Yr-1: ', ysic2.shape)
print(ysic2[:,:,0])
print('')
print('')
print('')
print('input data of SIC MON-1: ', msic2.shape)
print(msic2[:,:,0])
print('')
print('')
print('')
print('input data of Anomaly SIC Yr-1: ', aysic2.shape)
print(aysic2[:,:,0])
print('')
print('')
print('')
print('input data of Anomaly SIC Mon-1: ', amsic2.shape)
print(amsic2[:,:,0])
print('')
print('')
print('')
print('input data of THK Mon-1: ', mthk2.shape)
print(mthk2[:,:,0])
print('')
print('')
print('')
print('input data of V10 Mon-1: ', mvtn2.shape)
print(mvtn2[:,:,0])
print('')
print('')
print('')
print('input data of TEMP Mon-1: ', mtem2.shape)
print(mtem2[:,:,0])
print('')
print('')
print('')
print('input data of ALBEDO Mon-1: ', malb2.shape)
print(malb2[:,:,0])

In [ ]:
Y_SIC3 = image.extract_patches_2d(Y_SIC2, (psize, psize))

ysic3  = image.extract_patches_2d(ysic2,  (psize, psize))
ythk3  = image.extract_patches_2d(ythk2,  (psize, psize))
yvtn3  = image.extract_patches_2d(yvtn2,  (psize, psize))
ytem3  = image.extract_patches_2d(ytem2,  (psize, psize))
yalb3  = image.extract_patches_2d(yalb2,  (psize, psize))

msic3  = image.extract_patches_2d(msic2,  (psize, psize))
mthk3  = image.extract_patches_2d(mthk2,  (psize, psize))
mvtn3  = image.extract_patches_2d(mvtn2,  (psize, psize))
mtem3  = image.extract_patches_2d(mtem2,  (psize, psize))
malb3  = image.extract_patches_2d(malb2,  (psize, psize))

amsic3 = image.extract_patches_2d(amsic2, (psize, psize))
amthk3 = image.extract_patches_2d(amthk2, (psize, psize))
amvtn3 = image.extract_patches_2d(amvtn2, (psize, psize))
amtem3 = image.extract_patches_2d(amtem2, (psize, psize))
amalb3 = image.extract_patches_2d(amalb2, (psize, psize))

aysic3 = image.extract_patches_2d(aysic2, (psize, psize))
aythk3 = image.extract_patches_2d(aythk2, (psize, psize))
ayvtn3 = image.extract_patches_2d(ayvtn2, (psize, psize))
aytem3 = image.extract_patches_2d(aytem2, (psize, psize))
ayalb3 = image.extract_patches_2d(ayalb2, (psize, psize))

In [ ]:
print(" ysic3 shape:", ysic3.shape)
print(" msic3 shape:", msic3.shape)
print("aysic3 shape:", aysic3.shape)
print("amsic3 shape:", amsic3.shape)

In [ ]:
Y_SIC  = np.transpose(Y_SIC3,(1,2,3,0))

ysic   = np.transpose(ysic3,(1,2,3,0))
ythk   = np.transpose(ythk3,(1,2,3,0))
yvtn   = np.transpose(yvtn3,(1,2,3,0))
ytem   = np.transpose(ytem3,(1,2,3,0))
yalb   = np.transpose(yalb3,(1,2,3,0))

msic   = np.transpose(msic3,(1,2,3,0))
mthk   = np.transpose(mthk3,(1,2,3,0))
mvtn   = np.transpose(mvtn3,(1,2,3,0))
mtem   = np.transpose(mtem3,(1,2,3,0))
malb   = np.transpose(malb3,(1,2,3,0))

aysic   = np.transpose(aysic3,(1,2,3,0))
aythk   = np.transpose(aythk3,(1,2,3,0))
ayvtn   = np.transpose(ayvtn3,(1,2,3,0))
aytem   = np.transpose(aytem3,(1,2,3,0))
ayalb   = np.transpose(ayalb3,(1,2,3,0))

amsic   = np.transpose(amsic3,(1,2,3,0))
amthk   = np.transpose(amthk3,(1,2,3,0))
amvtn   = np.transpose(amvtn3,(1,2,3,0))
amtem   = np.transpose(amtem3,(1,2,3,0))
amalb   = np.transpose(amalb3,(1,2,3,0))


print("Y_SIC.shape:", Y_SIC.shape)
print("ysic.shape:", ysic.shape)
print("msic.shape:", msic.shape)

In [ ]:
px, py, nyr, npatch = ysic.shape

aysic_2d = aysic.reshape((px*py, nyr*npatch))
amsic_2d = amsic.reshape((px*py, nyr*npatch))
mthk_2d  =  mthk.reshape((px*py, nyr*npatch))
mvtn_2d  =  mvtn.reshape((px*py, nyr*npatch))
malb_2d  =  malb.reshape((px*py, nyr*npatch))
mtem_2d  =  mtem.reshape((px*py, nyr*npatch))


print("aysic_2d.shape:", aysic_2d.shape)
print("amsic_2d.shape:", amsic_2d.shape)
print("mthk_2d.shape :", mthk_2d.shape)
print("mvtn_2d.shape :", mvtn_2d.shape)
print("malb_2d.shape :", malb_2d.shape)
print("mtem_2d.shape :", mtem_2d.shape)

In [ ]:
#normalization
scaler = StandardScaler()

naysic_2d = scaler.fit_transform(aysic_2d)
namsic_2d = scaler.fit_transform(amsic_2d)
nmthk_2d  = scaler.fit_transform(mthk_2d)
nmvtn_2d  = scaler.fit_transform(mvtn_2d)
nmalb_2d  = scaler.fit_transform(malb_2d)
nmtem_2d  = scaler.fit_transform(mtem_2d)
nysic     =  ysic.reshape((px, py, nyr*npatch))/100
nmsic     =  msic.reshape((px, py, nyr*npatch))/100

print("naysic_2d.shape:", naysic_2d.shape)
print("namsic_2d.shape:", namsic_2d.shape)
print("nmthk_2d.shape :", nmthk_2d.shape)
print("nmvtn_2d.shape :", nmvtn_2d.shape)
print("nmalb_2d.shape :", nmalb_2d.shape)
print("nmtem_2d.shape :", nmtem_2d.shape)

In [ ]:
Y_SIC_Center = Y_SIC[cx,cy,:,:].reshape((nyr*npatch))

naysic = naysic_2d.reshape((px, py, nyr*npatch))
namsic = namsic_2d.reshape((px, py, nyr*npatch))
nmthk  =  nmthk_2d.reshape((px, py, nyr*npatch))
nmvtn  =  nmvtn_2d.reshape((px, py, nyr*npatch))
nmalb  =  nmalb_2d.reshape((px, py, nyr*npatch))
nmtem  =  nmtem_2d.reshape((px, py, nyr*npatch))

print("Y_SIC_Center.shape:", Y_SIC_Center.shape)

print("nysic.shape: ",  nysic.shape)
print("nmsic.shape: ",  nmsic.shape)
print("naysic.shape:", naysic.shape)
print("namsic.shape:", namsic.shape)
print("nmthk.shape: ",  nmthk.shape)
print("nmvtn.shape: ",  nmthk.shape)
print("nmalb.shape: ",  nmalb.shape)
print("nmtem.shape: ",  nmtem.shape)

print("nysic.range:  ",nysic.min(), nysic.max())
print("nmsic.range:  ",nmsic.min(), nmsic.max())
print("naysic.range: ",naysic.min(), naysic.max())
print("namsic.range: ",namsic.min(), namsic.max())
print("nmthk.range:  ",nmthk.min(), nmthk.max())
print("nmvtn.range:  ",nmthk.min(), nmthk.max())
print("nmalb.range:  ",nmalb.min(), nmalb.max())
print("nmtem.range:  ",nmtem.min(), nmtem.max())

In [ ]:
# These are the prediction target of SIC
plt.plot(Y_SIC_Center[85000:86000])
plt.title("Samples of Target SIC")
print("size of Y_SIC_Center: ",Y_SIC_Center.shape)

In [ ]:
# Combine all the standardized data and reshape
X1 = np.stack((nysic,nmsic,naysic,namsic,nmthk,nmvtn,nmalb,nmtem))
print("X1.shape:", X1.shape)
X  = np.transpose(X1)
print("X.shape:", X.shape)

In [ ]:
# Split the data into training (first 90%) and testing (last 10%)
train_num = int(ptrain*nyr*npatch)

train_X = X[:train_num,:,:,:]
test_X  = X[train_num:,:,:,:]
train_Y = Y_SIC_Center[:train_num]
test_Y  = Y_SIC_Center[train_num:]

print("train_X.shape: ",  train_X.shape)
print("train_Y.shape: ",  train_Y.shape)
print("test_X.shape:  ",  test_X.shape)
print("test_Y.shape:  ",  test_Y.shape)

In [ ]:
# 3-layer CNN model with LeakyReLU as activation function and 0.25 Dropout rate 
model = Sequential()
model.add(Conv2D(128, kernel_size=(5, 5),activation='relu',input_shape=(11,11,8),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((5, 5),padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(5, 5),padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(1, activation='relu'))
model.compile(optimizer="adam", loss="mse", metrics=['accuracy','mse'])

In [ ]:
model.summary()

In [ ]:
# randomize the training data (first 90%)
XTraining, XValidation, YTraining, YValidation = train_test_split(train_X,train_Y,test_size=0.1)

In [ ]:
# model training with 100 epochs
model_train = model.fit(XTraining,YTraining, batch_size=batch_size,epochs=epochs,verbose=2,validation_data=(XValidation,YValidation))

In [ ]:
model.save(h5_model)

In [ ]:
model_train.history

In [ ]:
accuracy = model_train.history['accuracy']
val_accuracy = model_train.history['val_accuracy']
mse = model_train.history['mse']
val_mse = model_train.history['val_mse']
loss = model_train.history['loss']
val_loss = model_train.history['val_loss']
epochs = range(len(accuracy))
plt.subplot(121)
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.subplot(122)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig(pg_train, dpi=150)
plt.close()

In [ ]:
# model prediction with test data (last 10%)
predicted_SIC = model.predict(test_X)
predicted_SIC.shape

In [ ]:
# These are the prediction target of SIC
plt.subplot(121)
plt.plot(test_Y[:500], 'b', label='Target SIC')
plt.plot(predicted_SIC[:500,0], 'r', label='Predicted SIC')
plt.title("Target vs. Predicted SIC")
plt.legend(loc='lower left')
# These are the prediction target of SIC
plt.subplot(122)
plt.plot(predicted_SIC[:500,0]-test_Y[:500], 'r', label='Predicted SIC')
plt.title("Predicted - Targeted SIC")
plt.savefig(pg_predt, dpi=150)
plt.close()